In [73]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [74]:
!wget https://storage.googleapis.com/tensorflow-1-public/course2/week3/horse-or-human.zip

--2022-08-06 10:10:08--  https://storage.googleapis.com/tensorflow-1-public/course2/week3/horse-or-human.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.212.128, 172.217.214.128, 172.253.114.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.212.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 149574867 (143M) [application/zip]
Saving to: ‘horse-or-human.zip.1’

horse-or-human.zip. 100%[===================>] 142.65M   114MB/s    in 1.3s    

2022-08-06 10:10:09 (114 MB/s) - ‘horse-or-human.zip.1’ saved [149574867/149574867]



In [75]:
import zipfile

# Unzip the dataset
local_zip = './horse-or-human.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('./horse-or-human')
zip_ref.close()

In [76]:
train_dir = './horse-or-human'
validation_dir = './horse-or-human-validation'

In [77]:
model = keras.Sequential([
    keras.layers.Conv2D(filters=16, kernel_size=(3, 3), input_shape=[300, 300, 3], activation=tf.nn.relu),
    keras.layers.MaxPooling2D(2, 2),
    keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation=tf.nn.relu),
    keras.layers.MaxPooling2D(2, 2),
    keras.layers.Conv2D(filters=64, kernel_size=(3, 3), activation=tf.nn.relu),
    keras.layers.MaxPooling2D(2, 2),
    keras.layers.Conv2D(filters=64, kernel_size=(3, 3), activation=tf.nn.relu),
    keras.layers.MaxPooling2D(2, 2),
    keras.layers.Conv2D(filters=64, kernel_size=(3, 3), activation=tf.nn.relu),
    keras.layers.MaxPooling2D(2, 2),
    keras.layers.Flatten(),
    keras.layers.Dense(units=512, activation=tf.nn.relu),
    keras.layers.Dense(units=1, activation=tf.nn.sigmoid)
])

model.summary(line_length = 100)

Model: "sequential_3"
____________________________________________________________________________________________________
 Layer (type)                                Output Shape                            Param #        
 conv2d_11 (Conv2D)                          (None, 298, 298, 16)                    448            
                                                                                                    
 max_pooling2d_11 (MaxPooling2D)             (None, 149, 149, 16)                    0              
                                                                                                    
 conv2d_12 (Conv2D)                          (None, 147, 147, 32)                    4640           
                                                                                                    
 max_pooling2d_12 (MaxPooling2D)             (None, 73, 73, 32)                      0              
                                                                     

In [78]:
model.compile(optimizer=tf.optimizers.RMSprop(learning_rate=0.0001), loss=tf.losses.binary_crossentropy, metrics=['accuracy'])

In [79]:
train_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(300, 300),
    batch_size=128,
    class_mode='binary'
)

Found 1027 images belonging to 2 classes.


In [80]:
validation_datagen = ImageDataGenerator(rescale=1./255)
validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(300, 300),
    batch_size=32,
    class_mode='binary'
)

Found 256 images belonging to 2 classes.


In [81]:
history = model.fit(
    train_generator,
    steps_per_epoch=8,
    epochs=5,
    validation_data=validation_generator,
    validation_steps=8,
    verbose=1
)

Epoch 1/5
8/8 [==============================] - 91s 11s/step - loss: 0.6719 - accuracy: 0.5306 - val_loss: 0.6548 - val_accuracy: 0.5078
Epoch 2/5
8/8 [==============================] - 88s 11s/step - loss: 0.5815 - accuracy: 0.8053 - val_loss: 0.4352 - val_accuracy: 0.8672
Epoch 3/5
8/8 [==============================] - 88s 11s/step - loss: 0.4777 - accuracy: 0.8309 - val_loss: 0.4305 - val_accuracy: 0.8281
Epoch 4/5
8/8 [==============================] - 90s 11s/step - loss: 0.3283 - accuracy: 0.9299 - val_loss: 0.6328 - val_accuracy: 0.7930
Epoch 5/5
8/8 [==============================] - 89s 11s/step - loss: 0.2576 - accuracy: 0.9221 - val_loss: 0.6447 - val_accuracy: 0.8320


In [82]:
## CODE BLOCK FOR NON-SAFARI BROWSERS
## SAFARI USERS: PLEASE SKIP THIS BLOCK AND RUN THE NEXT ONE INSTEAD

import numpy as np
from google.colab import files
from keras.preprocessing import image

uploaded = files.upload()

for fn in uploaded.keys():
 
  # predicting images
  path = '/content/' + fn
  img = image.load_img(path, target_size=(300, 300))
  x = image.img_to_array(img)
  x /= 255
  x = np.expand_dims(x, axis=0)

  images = np.vstack([x])
  classes = model.predict(images, batch_size=10)
  print(classes[0])
    
  if classes[0]>0.5:
    print(fn + " is a human")
  else:
    print(fn + " is a horse")
 

Saving valhuman01-19.png to valhuman01-19.png
[0.9970952]
valhuman01-19.png is a human


In [ ]:
!mv ./horse-or-human-validation/human ./horse-or-human-validation/humans